In [ ]:
# KARAR DESTEK SİSTEMLERİ PROJESİ

## ÖĞRENCİ BİLGİLERİ
## Ad Soyad: Akif Keklik
## Numara: 222803008

In [ ]:
# ==== ADIM 1: GEREKLİ KÜTÜPHANELER ====
print("⏳ Kütüphaneler kuruluyor... (Mehmet Akif Keklik Projesi)")

!pip install -q transformers peft accelerate bitsandbytes trl
!pip install -q langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers gradio datasets huggingface_hub

print("✅ Kurulum tamamlandı.")

In [ ]:
# ==== ADIM 2: OTOMATİK GİRİŞ ====
from huggingface_hub import login

# BURAYA KENDİ TOKEN'INI YAZ (hf_ ile başlayan kod)
my_token = "hf_BURAYA_TOKENINI_YAPISTIR"

print("🔑 Hugging Face'e giriş yapılıyor...")

try:
    login(token=my_token)
    print("✅ BAŞARILI: Giriş yapıldı!")
except Exception as e:
    print(f"❌ HATA: Token hatalı. Lütfen kontrol et. Hata: {e}")

In [ ]:
# ==== ADIM 3: FINE-TUNE EĞİTİMİ (TEKNOLOJİ MAĞAZASI) ====
import torch
import pandas as pd
from datasets import Dataset
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig

print("🚀 Veri Seti Hazırlanıyor...")

# 1. VERİ SETİ (MAĞAZA SORULARI)
qa_data = [
    {"q": "İade süresi kaç gün?", "a": "Ürünleri teslim aldıktan sonra 14 gün içinde koşulsuz iade edebilirsiniz."},
    {"q": "Kargo ücreti ne kadar?", "a": "1000 TL ve üzeri alışverişlerde kargo ücretsizdir, altındaki siparişlerde 50 TL'dir."},
    {"q": "Garanti süresi ne kadar?", "a": "Tüm elektronik ürünlerimiz 2 yıl resmi distribütör garantilidir."},
    {"q": "Hangi kargo ile çalışıyorsunuz?", "a": "Siparişleriniz Yurtiçi Kargo veya Aras Kargo ile gönderilmektedir."},
    {"q": "Mağazadan teslim alabilir miyim?", "a": "Evet, 'Mağazadan Gel Al' seçeneği ile siparişinizi aynı gün teslim alabilirsiniz."},
    {"q": "Taksit imkanı var mı?", "a": "Anlaşmalı kredi kartlarına vade farksız 3 taksit, vadeli 12 taksit imkanı vardır."},
    {"q": "Ürün bozuk çıktı ne yapmalıyım?", "a": "Arızalı ürünler için 14 gün içinde değişim, sonrasında ise yetkili servise yönlendirme yapılır."},
    {"q": "Siparişim ne zaman kargolanır?", "a": "Saat 16:00'ya kadar verilen siparişler aynı gün kargoya verilir."},
    {"q": "Öğrenci indirimi var mı?", "a": "Evet, EduTR kartı olan öğrencilere seçili bilgisayarlarda %10 indirim uygulanır."},
    {"q": "Hediye paketi yapıyor musunuz?", "a": "Evet, sipariş notuna eklerseniz ücretsiz hediye paketi yapıyoruz."},
    # Modeli güçlendirmek için tekrarlar
    {"q": "İade süresi kaç gün?", "a": "Ürünleri teslim aldıktan sonra 14 gün içinde koşulsuz iade edebilirsiniz."},
    {"q": "Garanti süresi ne kadar?", "a": "Tüm elektronik ürünlerimiz 2 yıl resmi distribütör garantilidir."},
    {"q": "Kargo ücreti ne kadar?", "a": "1000 TL ve üzeri alışverişlerde kargo ücretsizdir, altındaki siparişlerde 50 TL'dir."},
]

def format_data_function(example):
    return f"### Talimat:\n{example['q']}\n\n### Cevap:\n{example['a']}"

df = pd.DataFrame(qa_data)
df["text"] = df.apply(format_data_function, axis=1)
dataset = Dataset.from_pandas(df)

# 2. MODEL YÜKLEME
model_name = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, torch_dtype=torch.bfloat16, device_map="auto"
)
model.config.use_cache = False
peft_config = LoraConfig(lora_alpha=16, lora_dropout=0.1, r=16, bias="none", task_type="CAUSAL_LM")

# 3. EĞİTİM BAŞLATMA
training_args = SFTConfig(
    output_dir="./results",
    num_train_epochs=50,  # 50 Tur Ezber
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=False, bf16=True, logging_steps=10, report_to="none"
)

trainer = SFTTrainer(
    model=model, train_dataset=dataset, peft_config=peft_config, args=training_args, formatting_func=lambda x: x["text"]
)

print("🚀 Eğitim Başlıyor (Tekno-Asistan)...")
trainer.train()
print("✅ Eğitim Bitti! Model mağaza kurallarını ezberledi.")

In [ ]:
# ==== ADIM 4: RAG SİSTEMİ (MAĞAZA POLİTİKALARI) ====
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

print("📚 RAG Sistemi Kuruluyor...")

# 1. KANIT METNİ (MAĞAZA KURALLARI)
rag_metni = """
İADE KOŞULLARI: Müşterilerimiz, ürünleri teslim aldıktan sonra 14 gün içinde koşulsuz iade etme hakkına sahiptir. İade edilecek ürünün kutusu açılmamış ve zarar görmemiş olmalıdır.
KARGO VE TESLİMAT: 1000 TL ve üzeri alışverişlerde kargo tamamen ücretsizdir. 1000 TL altındaki siparişlerde sabit 50 TL kargo ücreti alınır. Siparişler Yurtiçi veya Aras Kargo ile gönderilir.
GARANTİ SÜRELERİ: Satılan tüm elektronik ürünler (Telefon, Bilgisayar, Tablet vb.) 2 yıl boyunca resmi distribütör garantisi altındadır. Aksesuarların garantisi ise 6 aydır.
ÖĞRENCİ İNDİRİMİ: Geçerli bir öğrenci belgesi veya EduTR kartı ibraz eden öğrencilere, dizüstü bilgisayar kategorisinde %10 indirim uygulanır.
SERVİS VE ARIZA: Arızalı ürünler ilk 14 gün içinde birebir değiştirilir. 14 günü geçen durumlarda ürün yetkili servise gönderilir ve onarım süresi maksimum 20 iş günüdür.
ÖDEME SEÇENEKLERİ: Kredi kartlarına vade farksız 3 taksit yapılır. Havale/EFT ile ödemelerde %5 ekstra indirim uygulanır.
"""

# 2. VEKTÖR VERİTABANI
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_text(rag_metni)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_texts(texts, embeddings)

# 3. MODEL AYARLARI
model.eval()
text_generator = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, temperature=0.01, top_p=0.9, return_full_text=False
)

def rag_getir(soru):
    docs = vector_db.similarity_search(soru, k=2)
    return "\n".join([d.page_content for d in docs])

print("✅ RAG ve Model Hazır.")

In [ ]:
# ==== ADIM 5: FİNAL ARAYÜZ (AGRESİF GÜVENLİK MODU) ====
import gradio as gr

# 1. KATILAŞTIRILMIŞ RAG ARAMA
def rag_kanit_getir(soru):
    # k=1: Sadece en iyi sonucu getir
    sonuclar = vector_db.similarity_search_with_score(soru, k=1)

    if not sonuclar:
        return ""

    en_yakin_belge = sonuclar[0][0]
    mesafe_skoru = sonuclar[0][1] # 0'a ne kadar yakınsa o kadar iyi

    icerik = en_yakin_belge.page_content

    # --- KONSOL TAKİBİ (DEBUG) ---
    # Videoda görünmez ama aşağıda çalışırken siyah ekranda skoru gör diye yazdırıyorum.
    print(f"\n🔍 ANALİZ:\nSoru: {soru}\nBulunan: {icerik[:40]}...\nSkor: {mesafe_skoru:.4f}")

    # --- KORUMA 1: MESAFE EŞİĞİ (THRESHOLD) ---
    # Önceki 1.2 çok gevşekti. Bunu 0.9 yapıyoruz.
    # Eğer skor 0.9'dan büyükse (yani 1.0, 1.4 vs.) soru ile metin alakasızdır.
    if mesafe_skoru > 0.9:
        print("❌ REDDEDİLDİ: Skor çok yüksek (Alakasız)")
        return ""

    # --- KORUMA 2: KELİME ÇAKIŞMA KONTROLÜ (GARANTİ YÖNTEM) ---
    # Sadece sayı eşleşmesi yetmez (Örn: "50"). Metinde mantıklı bir kelime de geçmeli.
    # Sorudaki kelimeleri ayırıyoruz (vize, 50, aldım)
    soru_kelimeleri = set(soru.lower().split())
    dokuman_kelimeleri = set(icerik.lower().split())

    # Ortak kelime var mı bakıyoruz (stop words hariç basit kontrol)
    # Eğer sorudaki hiçbir kelime (sayılar hariç) dokümanda yoksa reddet.
    cakisma = soru_kelimeleri.intersection(dokuman_kelimeleri)

    # Eğer ortak kelime sayısı 0 ise ve skor sınırdayda risk alma.
    if not cakisma and mesafe_skoru > 0.6:
         print("❌ REDDEDİLDİ: Ortak kelime bulunamadı.")
         return ""

    print("✅ KABUL EDİLDİ.")
    return icerik

# 2. MODEL CEVAP (ROBOT MODU)
def model_cevap_ure(soru, kanit):
    if not kanit:
        # Burası çok önemli. Kanıt yoksa model asla çalıştırılmaz.
        return "Bu sorunun cevabı mağaza dokümanlarında bulunmamaktadır. Lütfen iade, garanti, kargo veya indirimler hakkında sorunuz."

    prompt = f"""### Talimat:
Sen bir mağaza asistanısın. Sadece aşağıdaki BİLGİ metnini kullanarak soruyu cevapla.
Dışarıdan ASLA bilgi ekleme. Yorum yapma.
Eğer sorunun cevabı BİLGİ içinde açıkça yazmıyorsa "Bilmiyorum" de.

BİLGİ:
{kanit}

SORU:
{soru}

### Cevap:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Temperature 0.01 = Sıfır Yaratıcılık
    # Repetition penalty = Tekrarı önler
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.01,
        do_sample=True,
        repetition_penalty=1.2
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Cevap:")[-1].strip()

# 3. ANALİZ FONKSİYONU
def analiz(soru):
    if not soru.strip(): return "", ""

    kanit = rag_kanit_getir(soru)
    cevap = model_cevap_ure(soru, kanit)

    if not kanit:
        gosterilecek_kanit = "⚠️ Soru, mağaza verileriyle eşleşmedi (Sistem tarafından engellendi)."
    else:
        gosterilecek_kanit = kanit

    return cevap, gosterilecek_kanit

# ==== TASARIM AYNI KALIYOR ====
theme = gr.themes.Soft(primary_hue="blue", secondary_hue="green", neutral_hue="gray").set(
    body_background_fill="#111827", block_background_fill="rgba(31, 41, 55, 0.7)", body_text_color="#f3f4f6", input_background_fill="rgba(55, 65, 81, 0.5)"
)
custom_css = "body { background: #111827; color: white !important; } .glass-card { background: rgba(255, 255, 255, 0.05); border: 1px solid rgba(255, 255, 255, 0.1); border-radius: 16px; }"

with gr.Blocks(theme=theme, css=custom_css, title="Mehmet Akif - Tekno Asistan") as demo:
    gr.HTML("<h1 style='text-align:center; color:white'>🛒 TEKNOLOJİ MAĞAZASI ASİSTANI</h1>")

    with gr.Row():
        input_box = gr.Textbox(label="Müşteri Sorusu", placeholder="Örn: İade süresi kaç gün?", elem_classes="glass-card", scale=4)
        btn = gr.Button("SORGULA", variant="primary", scale=1)

    with gr.Row():
        with gr.Column(elem_classes="glass-card"):
            gr.Markdown("### 🧠 YAPAY ZEKA (Cevap)")
            out_ft = gr.Textbox(show_label=False, lines=5)
        with gr.Column(elem_classes="glass-card"):
            gr.Markdown("### 📜 MAĞAZA POLİTİKASI (Kaynak)")
            out_rag = gr.Textbox(show_label=False, lines=5)

    btn.click(analiz, inputs=input_box, outputs=[out_ft, out_rag])
    input_box.submit(analiz, inputs=input_box, outputs=[out_ft, out_rag])

print("🚀 Güvenli Arayüz Başlatılıyor...")
demo.launch(share=True, debug=True)